Clipping an image, code

In [9]:
# import our necessary packages
import rasterio # rasterio helps us read the raster image file
from matplotlib import pyplot as plt # pyplot helps us plot/visualize our image data
import numpy as np # numpy helps us scale our underlying band values

# specify file to import
image_file = "20220629_054526_16_24a5_3B_AnalyticMS.tif"

# define it as a rasterio object so we can use rasterio functions
my_raster_image = rasterio.open(image_file)

# scale values for display purposes
def scale(band): 
    return band / 10000.0

# import our multiband layers - blue, green red
blue = scale(my_raster_image.read(1))
green = scale(my_raster_image.read(2))
red = scale(my_raster_image.read(3))

# stack them into a single image for a true color composite
my_image = np.dstack((red, green, blue))

# show the image using pylot
plt.imshow(my_image)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


MemoryError: Unable to allocate 3.79 GiB for an array with shape (9574, 13278, 4) and data type float64

<Figure size 640x480 with 1 Axes>

In [10]:
# get x and y bounds
xmin, ymin, xmax, ymax = my_raster_image.bounds

# Let's get the x and y range first 
x_range = xmax - xmin
y_range = ymax - ymin
print(round(x_range), round(y_range))

print("Initial values are {}, {}, {} and {}".format(
    round(xmin), 
    round(ymin), 
    round(xmax), 
    round(ymax))
)

# Respecify the maximum values to be smaller to get an AoI inside this existing image
xmin = 384000
ymin = 3077500
xmax = 400000
ymax = 3085243

# Check final values
print("Final values are {}, {}, {} and {}".format(
    round(xmin), 
    round(ymin), 
    round(xmax), 
    round(ymax))
)

39834 28722
Initial values are 376188, 3071016, 416022 and 3099738
Final values are 384000, 3077500, 400000 and 3085243


In [11]:
# define the geometry type, which is a polygon  
my_geojson = [{
	"type": "Polygon", 
	"coordinates": [ 
	  [
		[xmin, ymin],
		[xmax, ymin],
		[xmax, ymax],
		[xmin, ymax],
		[xmin, ymin]
	  ],
	]
  }]
my_geojson

[{'type': 'Polygon',
  'coordinates': [[[384000, 3077500],
    [400000, 3077500],
    [400000, 3085243],
    [384000, 3085243],
    [384000, 3077500]]]}]

In [12]:
# Masking is a technique used to clarify dense or detailed map content by having 
# the features of one layer hide, or mask, features of another layer where they overlap
from rasterio.mask import mask

In [13]:
# apply the rasterio mask 
# specify the function needs to crop (via crop=True)
with rasterio.open(image_file) as img:
    clipped, transform = mask(img, my_geojson, crop=True)
    
# view your clipped multiband numpy array
print(clipped)

[[[    0     0     0 ...     0     0     0]
  [10742 10788 10666 ... 10456 10720     0]
  [10756 10718 10720 ... 10437 10562     0]
  ...
  [10969 10971 10957 ... 10934 10870     0]
  [10960 10932 10933 ... 10855 10855     0]
  [10928 10865 10901 ... 10977 10899     0]]

 [[    0     0     0 ...     0     0     0]
  [11019 10887 10839 ... 10039 10113     0]
  [11052 10988 10768 ... 10006 10152     0]
  ...
  [10963 10845 11009 ... 10483 10367     0]
  [10936 10856 10913 ... 10609 10353     0]
  [10934 10925 10958 ... 10646 10358     0]]

 [[    0     0     0 ...     0     0     0]
  [10397 10414 10425 ...  9553  9680     0]
  [10480 10402 10341 ...  9374  9483     0]
  ...
  [10600 10693 10437 ... 10603 10314     0]
  [10589 10592 10608 ... 10650 10388     0]
  [10601 10430 10405 ... 10503 10368     0]]

 [[    0     0     0 ...     0     0     0]
  [ 9595  9932  9427 ...  8236  7977     0]
  [ 9510  9645  9824 ...  8504  8108     0]
  ...
  [ 8910  9075  8975 ...  9499  9126     0]
  

In [14]:
# copy the metadata from the original ratserio object
meta = my_raster_image.meta.copy()

# update metadata, and provide the new clipped boundaries
meta.update(
    {
    
        "transform": transform,
        "height":clipped.shape[1],
        "width":clipped.shape[2]
    }
)
print(meta) # show metadata

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': 0.0, 'width': 5334, 'height': 2582, 'count': 4, 'crs': CRS.from_epsg(32642), 'transform': Affine(3.0, 0.0, 384000.0,
       0.0, -3.0, 3085245.0)}


In [15]:
# Now we write new image to a GeoTIFF, 'w' is write
with rasterio.open('clipped_Jun_29.tif', 'w', **meta) as my_writer_object:
    my_writer_object.write(clipped)
    
print('Writing complete')

Writing complete
